In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

##cluster_name = "GIVE_A_CLUSTER_NAME"
cluster_name = "compute-cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print('Creating a new compute cluster...')
    # Specify the configuration for the new cluster
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    # Create the cluster with the specified name and configuration
    compute_target = ComputeTarget.create(workspace=ws,
                                        name=cluster_name, 
                                        provisioning_configuration=compute_config)

# Wait for the cluster to complete, show the output log
compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())


In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

######### MY CODE ########
import shutil
##########################

# Specify parameter sampler
##ps = ### YOUR CODE HERE ###
ps = RandomParameterSampling(
        {
            '--C': choice(0.01, 0.1, 1.0, 10.0, 100.0),
            '--max_iter': choice(20, 50, 100, 120, 150)
        }
    )

# Specify a Policy
##policy = ### YOUR CODE HERE ###
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

######### MY CODE ########
train_script = "./training"
shutil.copy('train.py', train_script)
##########################

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
##src = ### YOUR CODE HERE ###
src = ScriptRunConfig(source_directory=train_script, 
					script='train.py', 		
                    compute_target=compute_target, 	
                    environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
##hyperdrive_config = ### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(run_config=src, 									
                                    hyperparameter_sampling=ps, 									
                                    policy=policy, 											
                                    primary_metric_name='validation_acc',               # Accuracy						
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 				 		
                                    max_total_runs=4, 		                            # 20, 100						
                                    max_concurrent_runs=4) 			                    # 20

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###

# Start the HyperDrive run
hyperdrive_run = exp.submit(config = hyperdrive_config, show_output = True)

# Monitor HyperDrive runs You can monitor the progress of the runs with the following Jupyter widget
RunDetails(hyperdrive_run).show()

hyperdrive_run.wait_for_completion(show_output=True)

# evaluate the the run is indeed complete
assert(hyperdrive_run.get_status() == "Completed")

In [ ]:
import joblib
# Get your best run and save the model from that run.

######### MY CODE ########
# get_children_sorted_by_primary_metric: Returns a list of children sorted by their best primary metric.
# Each child in the result has run id, hyperparameters, best primary metric value and status.
print(hyperdrive_run.get_children_sorted_by_primary_metric(top=0, reverse=False, discard_no_metric=False))  ## top = 1
##########################

### YOUR CODE HERE ###

# Returns the best Run, or None if no child has the primary metric.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run

# get_metrics()
# Returns the metrics from all the runs that were launched by this HyperDriveRun.
print("Best run metrics :",best_run.get_metrics())

best_run_metrics = best_run.get_metrics()
print('Accuracy:', best_run_metrics['Accuracy'])

# check metrics details
best_run_metrics

# get_details()
# Returns a dictionary with the details for the run
print("Best run details :",best_run.get_details())
print(best_run.get_details()['runDefinition']['arguments'])

# get_file_names()
# Returns a list of the files that are stored in association with the run.
print("Best run file names :",best_run.get_file_names()) # get name of files of best_run

# download the best run and register the model
best_run.download_file(name=train_script+'/model.joblib', 								
                        output_file_path=train_script)

# save the model, i.e., output file of best_run
model = best_run.register_model(model_name='hyperdrive_run', 								
                                model_path=train_script+'/model.joblib')

# save file
os.makedirs('./outputs',exist_ok = True)
joblib.dump(value=best_run, filename = 'outputs/bestHDModel.txt')
joblib.dump(value=best_run, filename='outputs/bankmarketing_model.pkl')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(### YOUR DATA OBJECT HERE ###)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###